<a href="https://colab.research.google.com/github/thinothw/DFDS-Final-Project/blob/feature%2Fff%2B%2B---sandbox/Phase01_Sandbox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Phase 01 - Setting up the Environment.

In [1]:
!pip install retina-face opencv-python imagehash

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.7/296.7 kB 11.6 MB/s eta 0:00:00


In [2]:
# Clean, compatible install block
!pip uninstall -y numpy -q
!pip install -q numpy==1.26.4
!pip install -q datasets==2.18.0
!pip install -q facenet-pytorch==2.5.3
!pip install -q opencv-python==4.8.0.76

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 36.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
xarray-einstats 0.10.0 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
rasterio 1.5.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-contrib-python 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.13.0.92 requires numpy>=2; python_version >= "3.9", but you 

In [1]:
# INFRASTRUCTURE - Connect to Drive

from google.colab import drive
import os

print("Requesting Google Drive access...")
drive.mount('/content/drive')

# Double check the project folder exists
base_path = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K'
if os.path.exists(base_path):
    print(f"✅ Connection Stable! Project folder found at: {base_path}")
else:
    print(f"⚠️ Warning: Base path not found. Check your Drive folder name!")

Requesting Google Drive access...
Mounted at /content/drive
✅ Connection Stable! Project folder found at: /content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K


In [2]:
import numpy
import pandas
import torch
print("✅ All core imports successful. The environment is stable.")

✅ All core imports successful. The environment is stable.


Phase 02 - Downloading Data from OpenFake.

In [ ]:
# Phase 2 - Download.
# OpenFake Sandbox Dataset Downloader - 12,000 Images.
# Code ran with 0 saved errors!.

# 1. Core Imports
import os
import torch
import shutil
from datasets import load_dataset

# 2. GPU Verification
print("\nChecking GPU availability...")
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device detected: {device}")

# 3. Create High-Speed Local Folders
print("\nSetting up high-speed local storage...")
local_base = '/content/temp_raw'
local_real = os.path.join(local_base, 'real')
local_fake = os.path.join(local_base, 'fake')

os.makedirs(local_real, exist_ok=True)
os.makedirs(local_fake, exist_ok=True)

# 4. Connect to OpenFake Dataset
print("\nConnecting to OpenFake dataset stream...")
dataset_name = "ComplexDataLab/OpenFake"

try:
    openfake = load_dataset(dataset_name, split='train', streaming=True)
    print("Dataset connection successful.")
except Exception as e:
    raise RuntimeError(f"Dataset loading failed: {e}")

# 5. Download Buffer (6,000 Real + 6,000 Fake)
TARGET_BUFFER = 6000
real_count = 0
fake_count = 0
save_errors = 0
max_iterations = 80000
iteration_counter = 0

print(f"\nStarting MASS DOWNLOAD to local storage (Target: {TARGET_BUFFER} per class)...")

for item in openfake:
    iteration_counter += 1

    if iteration_counter > max_iterations:
        print("\n⚠️ Iteration cap exceeded. Stopping download.")
        break

    try:
        label = item['label']
        image = item.get('image', None) # Safely attempt to get the image

        # Safety 1: The Null Check
        if image is None:
            continue

        # Safety 2: Handle Integer Labels
        if isinstance(label, int):
            label = 'real' if label == 0 else 'fake'

        if label not in ['real', 'fake']:
            continue

        # Safety 3: Force RGB format
        if image.mode != 'RGB':
            image = image.convert('RGB')

        if label == 'real' and real_count < TARGET_BUFFER:
            image.save(os.path.join(local_real, f"raw_openfake_real_{real_count}.jpg"))
            real_count += 1
            if real_count % 1000 == 0:
                print(f"  -> Downloaded {real_count} / {TARGET_BUFFER} Real images...")

        elif label == 'fake' and fake_count < TARGET_BUFFER:
            image.save(os.path.join(local_fake, f"raw_openfake_fake_{fake_count}.jpg"))
            fake_count += 1
            if fake_count % 1000 == 0:
                print(f"  -> Downloaded {fake_count} / {TARGET_BUFFER} Fake images...")

        if real_count == TARGET_BUFFER and fake_count == TARGET_BUFFER:
            break

    except Exception as e:
        save_errors += 1
        print(f"Error at iteration {iteration_counter}: {e}")

print("\n=== Download Summary ===")
print(f"Real images saved locally: {real_count}")
print(f"Fake images saved locally: {fake_count}")
print(f"Save errors: {save_errors}")

# 6. Move to Google Drive
print("\nMoving files from local storage to Google Drive. Please wait...")
drive_base = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K/raw_data'
drive_real = os.path.join(drive_base, 'real')
drive_fake = os.path.join(drive_base, 'fake')

os.makedirs(drive_real, exist_ok=True)
os.makedirs(drive_fake, exist_ok=True)

# Copy everything over
shutil.copytree(local_real, drive_real, dirs_exist_ok=True)
shutil.copytree(local_fake, drive_fake, dirs_exist_ok=True)

# 7. Purge Local Temp Files
print("Cleaning up temporary local files to free space...")
shutil.rmtree(local_base)

print(" Mass download, Drive transfer, and cleanup completed successfully!")


Checking GPU availability...
Device detected: cpu

Setting up high-speed local storage...

Connecting to OpenFake dataset stream...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/206 [00:00<?, ?it/s]

Dataset connection successful.

Starting MASS DOWNLOAD to local storage (Target: 6000 per class)...
  -> Downloaded 1000 / 6000 Fake images...
  -> Downloaded 1000 / 6000 Real images...
  -> Downloaded 2000 / 6000 Real images...
  -> Downloaded 2000 / 6000 Fake images...


/usr/local/lib/python3.12/dist-packages/PIL/Image.py:1047: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


  -> Downloaded 3000 / 6000 Real images...
  -> Downloaded 3000 / 6000 Fake images...
  -> Downloaded 4000 / 6000 Fake images...
  -> Downloaded 4000 / 6000 Real images...
  -> Downloaded 5000 / 6000 Fake images...
  -> Downloaded 5000 / 6000 Real images...
  -> Downloaded 6000 / 6000 Real images...
  -> Downloaded 6000 / 6000 Fake images...

=== Download Summary ===
Real images saved locally: 6000
Fake images saved locally: 6000
Save errors: 0

Moving files from local storage to Google Drive. Please wait...
Cleaning up temporary local files to free space...
 Mass download, Drive transfer, and cleanup completed successfully!


Phase 02.1 - Running RetinaFace Bouncer Script on OpenFake Data.

In [ ]:
# PHASE 2 - RetinaFace Bouncer V3.1.1
# Load to Colab Local Drive then Offload to Drive.
# Live Updates of the Progress.


import os
import cv2
import numpy as np
import shutil
from retinaface import RetinaFace
from PIL import Image
import warnings
from tqdm import tqdm  # The real time progress bar

warnings.filterwarnings("ignore")

print("Bouncer operating on: GPU (RetinaFace V4.3.2 High-Speed + Live Tracking)\n")

# 1. Define Drive & Local Paths
drive_base = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K'
local_base = '/content/temp_workspace'

drive_raw_real = os.path.join(drive_base, 'raw_data/real')
drive_raw_fake = os.path.join(drive_base, 'raw_data/fake')

local_raw_real = os.path.join(local_base, 'raw_data/real')
local_raw_fake = os.path.join(local_base, 'raw_data/fake')

local_hq_real = os.path.join(local_base, 'processed_data/real')
local_hq_fake = os.path.join(local_base, 'processed_data/fake')
local_reject_real = os.path.join(local_base, 'rejected/real')
local_reject_fake = os.path.join(local_base, 'rejected/fake')

# 2. Teleport Raw Data to Local SSD for Speed
print(" Step 1: Teleporting raw data from Google Drive to Local SSD (This takes 1-2 mins)...")
os.makedirs(local_base, exist_ok=True)
shutil.copytree(drive_raw_real, local_raw_real, dirs_exist_ok=True)
shutil.copytree(drive_raw_fake, local_raw_fake, dirs_exist_ok=True)
print(" Local transfer complete! Setting up output folders...")

for path in [local_hq_real, local_hq_fake, local_reject_real, local_reject_fake]:
    os.makedirs(path, exist_ok=True)

# 3. Blur Detection Function
def blur_score(pil_image):
    img = np.array(pil_image)
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
    return cv2.Laplacian(gray, cv2.CV_64F).var()

# 4. RetinaFace Extraction Engine
def hq_crop_engine_v4_3(img_path, save_path, reject_path, padding=25, min_face_size=30, min_face_ratio=0.005, blur_threshold=50, confidence_threshold=0.90):
    try:
        faces = RetinaFace.detect_faces(img_path)
        if not isinstance(faces, dict) or len(faces) == 0:
            return False, "No Face Detected"

        largest_area = 0
        best_face = None
        for face in faces.values():
            box = face.get('facial_area', None)
            if box is None: continue
            area = (box[2] - box[0]) * (box[3] - box[1])
            if area > largest_area:
                largest_area = area
                best_face = face

        if best_face is None: return False, "No Valid Face Found"

        box = best_face['facial_area']
        confidence = best_face['score']
        if confidence < confidence_threshold: return False, f"Low Confidence ({confidence:.2f})"

        img_pil = Image.open(img_path).convert('RGB')
        width, height = img_pil.size
        face_w, face_h = box[2] - box[0], box[3] - box[1]

        if face_w < min_face_size or face_h < min_face_size: return False, "Too Small Pixels"
        if (face_w * face_h) / (width * height) < min_face_ratio: return False, "Too Small Ratio"

        x_min, y_min = max(0, int(box[0]) - padding), max(0, int(box[1]) - padding)
        x_max, y_max = min(width, int(box[2]) + padding), min(height, int(box[3]) + padding)

        cropped_img = img_pil.crop((x_min, y_min, x_max, y_max))
        blur_val = blur_score(cropped_img)

        if blur_val < blur_threshold: return False, "Blurred"

        cropped_img.save(save_path)
        return True, "Success"
    except Exception as e:
        return False, str(e)

# 5. High-Speed Cleaner with Live Progress Bar
def clean_dataset_fast(input_folder, output_folder, reject_folder, label, target_valid=2000):
    print(f"\n---  Processing {label} Folder (Target: {target_valid}) ---")

    images = [f for f in os.listdir(input_folder) if f.endswith(('.jpg', '.jpeg', '.png'))]
    success, rejected = 0, 0
    reasons = {}

    # The tqdm wrapper creates the live progress bar
    pbar = tqdm(total=target_valid, desc=f"Extracting {label} Faces", unit="face")

    for name in images:
        if success >= target_valid:
            break

        img_path = os.path.join(input_folder, name)
        save_path = os.path.join(output_folder, name)
        reject_path = os.path.join(reject_folder, name)

        ok, msg = hq_crop_engine_v4_3(img_path, save_path, reject_path)

        if ok:
            success += 1
            pbar.update(1) # Ticks the progress bar forward
        else:
            rejected += 1
            reasons[msg] = reasons.get(msg, 0) + 1
            try:
                Image.open(img_path).convert('RGB').save(reject_path)
            except:
                pass

    pbar.close()
    print(f"\n STATS for {label}: {success} Valid | {rejected} Rejected")

# 6. Execute the Run
clean_dataset_fast(local_raw_real, local_hq_real, local_reject_real, "REAL", target_valid=2000)
clean_dataset_fast(local_raw_fake, local_hq_fake, local_reject_fake, "FAKE", target_valid=2000)

# 7. Push Final Data Back to Drive
print("\n Step 3: Pushing pristine extracted faces back to Google Drive...")
drive_final_hq = os.path.join(drive_base, 'processed_data_V4_3')
shutil.copytree(os.path.join(local_base, 'processed_data'), drive_final_hq, dirs_exist_ok=True)

# 8. Clean Up
print("🧹 Sweeping temporary local files...")
shutil.rmtree(local_base)

print("\n OpenFake Phase Complete! 4,000 perfectly balanced images locked into your Drive.")

Bouncer operating on: GPU (RetinaFace V4.3.2 High-Speed + Live Tracking)

 Step 1: Teleporting raw data from Google Drive to Local SSD (This takes 1-2 mins)...
 Local transfer complete! Setting up output folders...

---  Processing REAL Folder (Target: 2000) ---


Extracting REAL Faces:   0%|          | 0/2000 [00:00<?, ?face/s]

26-02-27 15:49:41 - Directory /root/.deepface created
26-02-27 15:49:41 - Directory /root/.deepface/weights created
26-02-27 15:49:41 - retinaface.h5 will be downloaded from the url https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5


Downloading...
From: https://github.com/serengil/deepface_models/releases/download/v1.0/retinaface.h5
To: /root/.deepface/weights/retinaface.h5

  0%|          | 0.00/119M [00:00<?, ?B/s]
 22%|██▏       | 25.7M/119M [00:00<00:00, 256MB/s]
 50%|████▉     | 59.2M/119M [00:00<00:00, 302MB/s]
 76%|███████▌  | 89.7M/119M [00:00<00:00, 270MB/s]
100%|██████████| 119M/119M [00:00<00:00, 268MB/s]
Extracting REAL Faces: 100%|██████████| 2000/2000 [20:02<00:00,  1.66face/s]



 STATS for REAL: 2000 Valid | 1240 Rejected

---  Processing FAKE Folder (Target: 2000) ---


Extracting FAKE Faces: 100%|██████████| 2000/2000 [11:24<00:00,  2.92face/s]



 STATS for FAKE: 2000 Valid | 1420 Rejected

 Step 3: Pushing pristine extracted faces back to Google Drive...
🧹 Sweeping temporary local files...

 OpenFake Phase Complete! 4,000 perfectly balanced images locked into your Drive.


Phase 03 - Downloading Data from FF++.

In [ ]:
import os
import urllib.request
import subprocess
import shutil
from google.colab import drive
from google.colab import userdata

# Unlock Vault
print("\nAccessing secure vault...")
TUM_URL = userdata.get('TUM_LINK')
if not TUM_URL:
    raise ValueError(" ERROR: 'TUM_LINK' not found in Secrets. Please check the spelling!")

# Fetch Downloader directly from secret TUM link
print("\nFetching official TUM downloader script from your secure link...")
urllib.request.urlretrieve(TUM_URL, "download.py")

# Define Paths
temp_base = '/content/ff_temp'
drive_base = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K/FF_Plus/Test_Run/raw_videos'
categories = ['original', 'Deepfakes', 'Face2Face', 'FaceSwap', 'NeuralTextures']

print("\n Firing up V3.2 auto-bypassing download sequence...")

for cat in categories:
    sub_folder = 'real' if cat == 'original' else f'fake/{cat.lower()}'
    temp_path = os.path.join(temp_base, sub_folder)
    drive_path = os.path.join(drive_base, sub_folder)

    shutil.rmtree(temp_path, ignore_errors=True)
    os.makedirs(temp_path, exist_ok=True)
    os.makedirs(drive_path, exist_ok=True)

    print(f"\n Pulling 2 test videos for: {cat} to high-speed local storage...")

    # THE FIX: Echo automatically presses Enter to agree to the TOS
    cmd = f'echo "" | python3 download.py {temp_path} -d {cat} -c c23 -n 2 --server EU2'
    result = subprocess.run(cmd, shell=True)

    if result.returncode != 0:
        print(f" ERROR: Download failed for {cat}. Check server status.")
    else:
        print(f" Download complete. Transferring {cat} to Google Drive...")
        shutil.copytree(temp_path, drive_path, dirs_exist_ok=True)
        print(f" Transfer complete for {cat}.")

print("\n Micro-Test Download Complete!")

Mounting Google Drive...
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Accessing secure vault...

Fetching official TUM downloader script from your secure link...

 Firing up V3.2 auto-bypassing download sequence...

 Pulling 2 test videos for: original to high-speed local storage...
 Download complete. Transferring original to Google Drive...
 Transfer complete for original.

 Pulling 2 test videos for: Deepfakes to high-speed local storage...
 Download complete. Transferring Deepfakes to Google Drive...
 Transfer complete for Deepfakes.

 Pulling 2 test videos for: Face2Face to high-speed local storage...
 Download complete. Transferring Face2Face to Google Drive...
 Transfer complete for Face2Face.

 Pulling 2 test videos for: FaceSwap to high-speed local storage...
 Download complete. Transferring FaceSwap to Google Drive...
 Transfer complete for FaceSwap.

 Pulling 2 test videos for: NeuralTexture

Phase 03.1 - Running the Frame Extracter for the FF++ Vidoes.

In [5]:
import os
import glob
import random
import shutil
import cv2
import hashlib
from tqdm import tqdm

# Lock the random seed for perfect reproducibility
random.seed(42)

drive_raw_base = "/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K/FF_Plus/Test_Run/raw_videos"
drive_extract_base = "/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K/FF_Plus/Test_Run/extracted_frames_v2"
local_temp_base = "/content/local_processing"

target_frames_with_buffer = 7
split_ratios = {"train": 0.70, "val": 0.15, "test": 0.15}

os.makedirs(local_temp_base, exist_ok=True)
local_video_path = os.path.join(local_temp_base, "processing_vid.mp4")

# Initialize Traceability Log
log_file_path = "/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K/FF_Plus/Test_Run/split_log.txt"
with open(log_file_path, "w") as f:
    f.write("FF++ Video Split and Extraction Log\n")
    f.write("===================================\n")

def extract_buffered_frames(video_path, output_folder, category_prefix, n_frames):
    cap = cv2.VideoCapture(video_path)
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    if total_frames < n_frames:
        cap.release()
        return 0, total_frames

    step = total_frames // n_frames
    start_offset = random.randint(0, max(0, step - 1))
    target_ids = set([start_offset + (i * step) for i in range(n_frames)])

    vid_name = os.path.basename(video_path).split('.')[0]

    # Generate a short hash to absolutely prevent filename collisions
    short_hash = hashlib.md5(video_path.encode()).hexdigest()[:6]

    extracted_count = 0
    current_id = 0

    while cap.isOpened():
        ret = cap.grab()
        if not ret:
            break

        if current_id in target_ids:
            ret, frame = cap.retrieve()
            if ret:
                frame_name = f"{category_prefix}_{vid_name}_{short_hash}_f{extracted_count:03d}.jpg"
                save_path = os.path.join(output_folder, frame_name)
                cv2.imwrite(save_path, frame)
                extracted_count += 1

        current_id += 1
        if extracted_count >= n_frames:
            break

    cap.release()
    return extracted_count, total_frames

print("Initiating reproducible extraction sequence...")
categories = ['real', 'fake/deepfakes', 'fake/face2face', 'fake/faceswap', 'fake/neuraltextures']

total_extracted = 0
skipped_videos = 0

for cat in categories:
    cat_path = os.path.join(drive_raw_base, cat)
    videos = glob.glob(os.path.join(cat_path, "**", "*.mp4"), recursive=True)

    if not videos:
        continue

    random.shuffle(videos)

    train_split = int(len(videos) * split_ratios["train"])
    val_split = int(len(videos) * (split_ratios["train"] + split_ratios["val"]))

    splits = {
        "train": videos[:train_split],
        "val": videos[train_split:val_split],
        "test": videos[val_split:]
    }

    for split_name, split_videos in splits.items():
        if not split_videos:
            continue

        cat_safe_name = cat.replace('/', '_')
        final_drive_folder = os.path.join(drive_extract_base, split_name, cat_safe_name)
        os.makedirs(final_drive_folder, exist_ok=True)

        pbar = tqdm(total=len(split_videos), desc=f"{split_name.upper()} - {cat_safe_name}", unit="vid")

        with open(log_file_path, "a") as log:
            for video_path in split_videos:
                # Overwrite directly to local SSD for maximum IO speed
                shutil.copy2(video_path, local_video_path)

                frames_saved, total_found = extract_buffered_frames(local_video_path, final_drive_folder, f"{split_name}_{cat_safe_name}", target_frames_with_buffer)

                if frames_saved == 0:
                    skipped_videos += 1
                    log.write(f"[SKIPPED] {video_path} (Only {total_found} frames)\n")
                else:
                    total_extracted += frames_saved
                    log.write(f"[{split_name.upper()}] {video_path} -> Extracted {frames_saved} frames\n")

                pbar.update(1)

        pbar.close()

shutil.rmtree(local_temp_base, ignore_errors=True)
print(f"\nExtraction complete. {total_extracted} frames stored.")
print(f"{skipped_videos} short videos skipped. Check split_log.txt for full traceability.")

Initiating reproducible extraction sequence...


TEST - fake_neuraltextures: 100%|██████████| 1/1 [00:03<00:00,  3.38s/vid]


Extraction complete. 70 frames stored.
0 short videos skipped. Check split_log.txt for full traceability.


Phase 03.2 - Running the RetinaFace on Extracted Frames.

In [5]:
import os
import cv2
import numpy as np
import shutil
import glob
import csv
import imagehash
import random
from retinaface import RetinaFace
from PIL import Image
import warnings
from tqdm import tqdm

warnings.filterwarnings("ignore")

# 1. Global Determinism
random.seed(42)
np.random.seed(42)

print("Master Bouncer Operating On: GPU (Hamming Dist pHash + Single I/O + BBox Logging)\n")

drive_input_base = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K/FF_Plus/Test_Run/extracted_frames_v2'
drive_output_base = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K/FF_Plus/Test_Run/processed_faces_v2'
drive_reject_base = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K/FF_Plus/Test_Run/rejected_faces_v2'
csv_log_path = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K/FF_Plus/Test_Run/extraction_log_v2.csv'

local_base = '/content/temp_workspace'
local_input = os.path.join(local_base, 'input_frames')
local_output = os.path.join(local_base, 'processed_faces')
local_reject = os.path.join(local_base, 'rejected_faces')

print("Teleporting buffered frames to Local SSD...")
shutil.rmtree(local_base, ignore_errors=True)
shutil.copytree(drive_input_base, local_input, dirs_exist_ok=True)

with open(csv_log_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(["Image_Name", "Split", "Category", "Status", "Reason", "Confidence", "Blur_Score", "pHash", "Bounding_Box"])

# Store actual hash objects for fast Hamming distance calculation
seen_hashes = []

def master_crop_engine(img_path, save_path, padding=25, min_face_size=30, min_face_ratio=0.005, blur_threshold=25, confidence_threshold=0.90):
    try:
        # 5. Single I/O Load: Read once with OpenCV
        img_cv = cv2.imread(img_path)
        if img_cv is None:
            return False, "Corrupted Image", 0.0, 0.0, "", "[]"

        height, width = img_cv.shape[:2]

        # RetinaFace accepts NumPy arrays directly
        faces = RetinaFace.detect_faces(img_cv)
        if not isinstance(faces, dict) or len(faces) == 0:
            return False, "No Face Detected", 0.0, 0.0, "", "[]"

        largest_area = 0
        best_face = None
        for face in faces.values():
            box = face.get('facial_area', None)
            if box is None: continue
            area = (box[2] - box[0]) * (box[3] - box[1])
            if area > largest_area:
                largest_area = area
                best_face = face

        if best_face is None:
            return False, "No Valid Face Found", 0.0, 0.0, "", "[]"

        box = best_face['facial_area']
        str_box = f"[{box[0]}, {box[1]}, {box[2]}, {box[3]}]"
        confidence = best_face['score']

        if confidence < confidence_threshold:
            return False, "Low Confidence", confidence, 0.0, "", str_box

        face_w, face_h = box[2] - box[0], box[3] - box[1]

        # 3. Reintroduced Face-to-Frame Ratio Filter
        if face_w < min_face_size or face_h < min_face_size:
            return False, "Resolution Too Small", confidence, 0.0, "", str_box
        if (face_w * face_h) / (width * height) < min_face_ratio:
            return False, "Face Ratio Too Small", confidence, 0.0, "", str_box

        # Fast NumPy cropping
        x_min, y_min = max(0, int(box[0]) - padding), max(0, int(box[1]) - padding)
        x_max, y_max = min(width, int(box[2]) + padding), min(height, int(box[3]) + padding)
        cropped_cv = img_cv[y_min:y_max, x_min:x_max]

        # Calculate blur on the cropped NumPy array
        gray_crop = cv2.cvtColor(cropped_cv, cv2.COLOR_BGR2GRAY)
        blur_val = cv2.Laplacian(gray_crop, cv2.CV_64F).var()

        if blur_val < blur_threshold:
            return False, "Motion Blur Detected", confidence, blur_val, "", str_box

        # Convert to PIL ONLY for the final resize and hash
        cropped_pil = Image.fromarray(cv2.cvtColor(cropped_cv, cv2.COLOR_BGR2RGB))
        standardized_img = cropped_pil.resize((224, 224), Image.BICUBIC)

        # 2. Upgraded pHash Logic (Hamming Distance <= 5)
        new_hash = imagehash.phash(standardized_img)

        for existing_hash in seen_hashes:
            if new_hash - existing_hash <= 2:
                return False, "Duplicate Face (Hamming Dist <= 2)", confidence, blur_val, str(new_hash), str_box

        seen_hashes.append(new_hash)

        standardized_img.save(save_path, format='JPEG', quality=95)
        return True, "Accepted", confidence, blur_val, str(new_hash), str_box

    except Exception as e:
        return False, f"Engine Error: {str(e)}", 0.0, 0.0, "", "[]"

print("Firing up RetinaFace Deduplication Engine...")
image_files = glob.glob(os.path.join(local_input, "**", "*.jpg"), recursive=True)

if not image_files:
    print("ERROR: No images found in local storage.")
else:
    success_count, reject_count = 0, 0
    pbar = tqdm(total=len(image_files), desc="Processing Frames", unit="img")

    with open(csv_log_path, mode='a', newline='') as log_file:
        csv_writer = csv.writer(log_file)

        for img_path in image_files:
            relative_path = os.path.relpath(os.path.dirname(img_path), local_input)
            path_parts = relative_path.split(os.sep)
            split_name = path_parts[0] if len(path_parts) > 0 else "unknown"
            cat_name = path_parts[1] if len(path_parts) > 1 else "unknown"

            out_folder = os.path.join(local_output, relative_path)
            rej_folder = os.path.join(local_reject, relative_path)
            os.makedirs(out_folder, exist_ok=True)
            os.makedirs(rej_folder, exist_ok=True)

            file_name = os.path.basename(img_path)
            save_path = os.path.join(out_folder, file_name)
            reject_path = os.path.join(rej_folder, file_name)

            passed, msg, conf, blur, phash_val, bbox = master_crop_engine(img_path, save_path)

            # 4. Store Bounding Boxes in CSV
            if passed:
                success_count += 1
                csv_writer.writerow([file_name, split_name, cat_name, "Accepted", "None", round(conf, 4), round(blur, 2), phash_val, bbox])
            else:
                reject_count += 1
                csv_writer.writerow([file_name, split_name, cat_name, "Rejected", msg, round(conf, 4), round(blur, 2), phash_val, bbox])
                try:
                    Image.open(img_path).convert('RGB').save(reject_path, format='JPEG', quality=95)
                except:
                    pass

            pbar.update(1)

    pbar.close()

    print("\n--- BOUNCER STATS ---")
    print(f"Pristine Faces Secured: {success_count}")
    print(f"Frames Rejected: {reject_count}")

print("Pushing standardized dataset and CSV logs back to Google Drive...")
shutil.copytree(local_output, drive_output_base, dirs_exist_ok=True)
shutil.copytree(local_reject, drive_reject_base, dirs_exist_ok=True)
shutil.rmtree(local_base, ignore_errors=True)

print("Pipeline Complete! Check extraction_log_v2.csv for full traceability.")

Master Bouncer Operating On: GPU (Hamming Dist pHash + Single I/O + BBox Logging)

Teleporting buffered frames to Local SSD...
Firing up RetinaFace Deduplication Engine...


Processing Frames: 100%|██████████| 70/70 [00:16<00:00,  4.32img/s]



--- BOUNCER STATS ---
Pristine Faces Secured: 53
Frames Rejected: 17
Pushing standardized dataset and CSV logs back to Google Drive...
Pipeline Complete! Check extraction_log_v2.csv for full traceability.


- Stats from RetinaFace Extraction

In [6]:
import pandas as pd

csv_path = '/content/drive/MyDrive/Deepfake_Honours_Project/Sandbox_10K/FF_Plus/Test_Run/extraction_log_v2.csv'
df = pd.read_csv(csv_path)

print("--- BOUNCER REJECTION BREAKDOWN ---")
rejections = df[df['Status'] == 'Rejected']
print(rejections['Reason'].value_counts())

--- BOUNCER REJECTION BREAKDOWN ---
Reason
Duplicate Face (Hamming Dist <= 2)    14
Motion Blur Detected                   3
Name: count, dtype: int64
